In [1]:
%pwd

'/Users/vatsalkhetan/Downloads/Chest_CT_Cancer/research'

In [2]:
import os

In [3]:
os.chdir("../")

In [4]:
import os
print("Current Working Directory:", os.getcwd())

Current Working Directory: /Users/vatsalkhetan/Downloads/Chest_CT_Cancer


In [5]:
%pwd

'/Users/vatsalkhetan/Downloads/Chest_CT_Cancer'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from cnnclassifer.constants import *
from cnnclassifer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH

In [8]:
from cnnclassifer.utils.common import read_yaml, create_directories

In [9]:
import os
print("Current Working Directory:", os.getcwd())
CONFIG_FILE_PATH = "config/config.yaml"

# Get the absolute path of config.yaml
absolute_path = os.path.abspath(CONFIG_FILE_PATH)
print("Absolute path of config.yaml:", absolute_path)

Current Working Directory: /Users/vatsalkhetan/Downloads/Chest_CT_Cancer
Absolute path of config.yaml: /Users/vatsalkhetan/Downloads/Chest_CT_Cancer/config/config.yaml


In [10]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=Path(CONFIG_FILE_PATH),
            params_filepath=Path(PARAMS_FILE_PATH)):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir, 
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config
    
    print(f"Attempting to load config from: {CONFIG_FILE_PATH}")

Attempting to load config from: config/config.yaml


In [11]:
import os
import zipfile
import gdown
from cnnclassifer import logger
from cnnclassifer.utils.common import get_size

In [12]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
     
    def download_file(self)-> str:
        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-11-06 23:22:14,214: INFO: common: YAML file 'config/config.yaml' loaded successfully.]
Loaded YAML content: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}}
Type of content: <class 'dict'>
[2024-11-06 23:22:14,216: INFO: common: YAML file 'params.yaml' loaded successfully.]
Loaded YAML content: {'param1': 'value1', 'param2': 'value2', 'nested_params': {'sub_param1': 'value3', 'sub_param2': 'value4'}}
Type of content: <class 'dict'>
[2024-11-06 23:22:14,218: INFO: common: created directory at: artifacts]
[2024-11-06 23:22:14,219: INFO: common: created directory at: artifacts/data_ingestion]
[2024-11-06 23:22:14,222: INFO: 3333675265: Downloading data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing into fil

Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY&confirm=t&uuid=30662d30-897d-4dfa-b3b3-e3170bb7c65c
To: /Users/vatsalkhetan/Downloads/Chest_CT_Cancer/artifacts/data_ingestion/data.zip
100%|██████████| 49.0M/49.0M [00:07<00:00, 6.95MB/s]

[2024-11-06 23:22:29,539: INFO: 3333675265: Downloaded data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing into file artifacts/data_ingestion/data.zip]
